#Load modules and dataset

In [2]:
!pip install tensorflow_decision_forests
!pip install wurlitzer
!pip install sweetviz

     |████████████████████████████████| 6.2MB 4.3MB/s 
     |████████████████████████████████| 15.1MB 365kB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn  as sns
import numpy as np
import csv
import zipfile
import sweetviz as sv
#from autoviz.AutoViz_Class import AutoViz_Class
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from wurlitzer import sys_pipes

path ='/content/dataset.zip'
zip_ref = zipfile.ZipFile(path,'r')
zip_ref.extractall('/content/')
zip_ref.close()
train = pd.read_csv('/content/dataset/train.csv')
test = pd.read_csv('/content/dataset/test.csv')
sub = pd.read_csv('/content/dataset/sample_submission.csv')

display(train.head())
display(train.isnull().sum())
test.isnull().sum()


,Customer ID,Name,Gender,Age,Income (USD),Income Stability,Profession,Type of Employment,Location,Loan Amount Request (USD),Current Loan Expenses (USD),Expense Type 1,Expense Type 2,Dependents,Credit Score,No. of Defaults,Has Active Credit Card,Property ID,Property Age,Property Type,Property Location,Co-Applicant,Property Price,Loan Sanction Amount (USD)
0,C-36995,Frederica Shealy,F,56,1933.05,Low,Working,Sales staff,Semi-Urban,72809.58,241.08,N,N,3.0,809.44,0,NaN,746,1933.05,4,Rural,1,119933.46,54607.18
1,C-33999,America Calderone,M,32,4952.91,Low,Working,NaN,Semi-Urban,46837.47,495.81,N,Y,1.0,780.40,0,Unpossessed,608,4952.91,2,Rural,1,54791.00,37469.98
2,C-3770,Rosetta Verne,F,65,988.19,High,Pensioner,NaN,Semi-Urban,45593.04,171.95,N,Y,1.0,833.15,0,Unpossessed,546,988.19,2,Urban,0,72440.58,36474.43
3,C-26480,Zoe Chitty,F,65,NaN,High,Pensioner,NaN,Rural,80057.92,298.54,N,Y,2.0,832.70,1,Unpossessed,890,NaN,2,Semi-Urban,1,121441.51,56040.54
4,C-23459,Afton Venema,F,31,2614.77,Low,Working,High skill tech staff,Semi-Urban,113858.89,491.41,N,Y,NaN,745.55,1,Active,715,2614.77,4,Semi-Urban,1,208567.91,74008.28


Customer ID                       0
Name                              0
Gender                           53
Age                               0
Income (USD)                   4576
Income Stability               1683
Profession                        0
Type of Employment             7270
Location                          0
Loan Amount Request (USD)         0
Current Loan Expenses (USD)     172
Expense Type 1                    0
Expense Type 2                    0
Dependents                     2493
Credit Score                   1703
No. of Defaults                   0
Has Active Credit Card         1566
Property ID                       0
Property Age                   4850
Property Type                     0
Property Location               356
Co-Applicant                      0
Property Price                    0
Loan Sanction Amount (USD)      340
dtype: int64

Customer ID                       0
Name                              0
Gender                           31
Age                               0
Income (USD)                    750
Income Stability                813
Profession                        0
Type of Employment             4689
Location                          0
Loan Amount Request (USD)         0
Current Loan Expenses (USD)      83
Expense Type 1                    0
Expense Type 2                    0
Dependents                     1142
Credit Score                    743
No. of Defaults                   0
Has Active Credit Card         1076
Property ID                       0
Property Age                    892
Property Type                     0
Property Location               160
Co-Applicant                      0
Property Price                    0
dtype: int64

#EDA

In [4]:
report = sv.analyze(train)
report.show_html('train.html')

                                             |          | [  0%]   00:00 -> (? left)

Report train.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [5]:
report = sv.analyze(test)
report.show_html('test.html')

                                             |          | [  0%]   00:00 -> (? left)

Report test.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [6]:
train.nunique()

Customer ID                    30000
Name                           30000
Gender                             2
Age                               48
Income (USD)                   24429
Income Stability                   2
Profession                         8
Type of Employment                18
Location                           3
Loan Amount Request (USD)      29982
Current Loan Expenses (USD)    24041
Expense Type 1                     2
Expense Type 2                     2
Dependents                        10
Credit Score                   17586
No. of Defaults                    2
Has Active Credit Card             3
Property ID                      999
Property Age                   24179
Property Type                      4
Property Location                  3
Co-Applicant                       3
Property Price                 29632
Loan Sanction Amount (USD)     21450
dtype: int64

# Dataset Preprocessing

In [7]:
a= train.copy()

# Replace values to get a cleaner dataste
a.replace({ '?':np.nan, -999 :-1, -999.0 :-1,'0' :0, '1':1 }, inplace=True) 
test.replace({ '?':np.nan, -999 :-1, -999.0 :-1,'0' :0, '1':1 }, inplace=True)


# 'Property Age' values are same as 'Income (USD)', also 274 fewer values. Others are ID columns
a.drop(columns=['Property Age','Customer ID','Name','Property ID'] , axis = 1, inplace=True) 
test.drop(columns=['Property Age','Customer ID','Name','Property ID'] , axis = 1, inplace=True)


# Remove index 20965 from train, clear outlier
a.drop(index= 20965, inplace=True) 


# Remove rows where target value is null 
y_nan = a[a['Loan Sanction Amount (USD)'].isnull() == 1]
a.drop(index= y_nan.index, inplace=True)
  
  
# Set null values in categorical col to 'unknown', as there are lots of null values. 
var_mod = a.select_dtypes(include='object').columns 
for ColName in var_mod:
  a[ColName] = np.where(a[ColName].isnull(),"Unknown",a[ColName])
  test[ColName] = np.where(test[ColName].isnull(),"Unknown",test[ColName])


# Labelencoding. Categorical values to int for train & test
le = LabelEncoder()
for i in var_mod:
  a[i] = le.fit_transform(a[i])
  test[i] = le.transform(test[i])

  

a = a.astype('float64') # turn encoded values from int to float
b = a.copy()
test = test.astype('float64')
d = test.copy()
a.drop(columns=['Loan Sanction Amount (USD)'],axis=1,inplace=True)
c = a.copy()



#Imputer to replace nan with similar values in train set
imp_mean = KNNImputer()
a = imp_mean.fit_transform(a)


# Scale the dataset 
scaler = StandardScaler()
a = scaler.fit_transform(a)
test = scaler.transform(test)

# Form pd df to visualize easily
a = pd.DataFrame(data = a, columns=c.columns,index = b.index)
test = pd.DataFrame(data=test, columns=d.columns, index = d.index)
a['target'] = b['Loan Sanction Amount (USD)']

df = a

df.isnull().sum() # No null values

Gender                         0
Age                            0
Income (USD)                   0
Income Stability               0
Profession                     0
Type of Employment             0
Location                       0
Loan Amount Request (USD)      0
Current Loan Expenses (USD)    0
Expense Type 1                 0
Expense Type 2                 0
Dependents                     0
Credit Score                   0
No. of Defaults                0
Has Active Credit Card         0
Property Type                  0
Property Location              0
Co-Applicant                   0
Property Price                 0
target                         0
dtype: int64

In [8]:
report = sv.analyze(df)
report.show_html('preprocess_df.html')

                                             |          | [  0%]   00:00 -> (? left)

Report preprocess_df.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


# Build Model and Predict

## Model 1

In [9]:
#shuffle dataset
train = df.sample(frac=1)

# Convert the Pandas dataframe to a tf dataset
target= "target"
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train,label=target, task=tfdf.keras.Task.REGRESSION)


model1 = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION,hyperparameter_template="benchmark_rank1")
model1.compile(metrics = ['r2'])
with sys_pipes():
  model1.fit(x = tf_dataset)

2021-07-09 09:41:52.906277: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-09 09:41:52.916326: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2299995000 Hz


464/464 [==============================] - 6s 2ms/step


[INFO kernel.cc:746] Start Yggdrasil model training
[INFO kernel.cc:747] Collect training examples
[INFO kernel.cc:392] Number of batches: 464
[INFO kernel.cc:393] Number of examples: 29659
[INFO kernel.cc:769] Dataset:
Number of records: 29659
Number of columns: 20

Number of columns by type:
	NUMERICAL: 20 (100%)

Columns:

NUMERICAL: 20 (100%)
	0: "Age" NUMERICAL mean:8.00443e-09 min:-1.37693 max:1.55144 sd:1
	1: "Co-Applicant" NUMERICAL mean:1.44495e-09 min:-4.88175 max:0.412622 sd:1
	2: "Credit_Score" NUMERICAL mean:7.89677e-11 min:-2.26367 max:2.21712 sd:1
	3: "Current_Loan_Expenses_(USD)" NUMERICAL mean:-1.66603e-10 min:-1.86244 max:15.6824 sd:1
	4: "Dependents" NUMERICAL mean:-9.64702e-09 min:-1.36103 max:12.763 sd:1
	5: "Expense_Type_1" NUMERICAL mean:3.38468e-08 min:-0.749932 max:1.33345 sd:1
	6: "Expense_Type_2" NUMERICAL mean:-2.59608e-08 min:-1.43324 max:0.697722 sd:1
	7: "Gender" NUMERICAL mean:-2.24078e-09 min:-1.00407 max:2.96841 sd:1
	8: "Has_Active_Credit_Card" NUMERI

In [10]:
x_test = tfdf.keras.pd_dataframe_to_tf_dataset(test)
pred1 = model1.predict(x_test)
pred1

array([[91595.73   ],
       [65274.227  ],
       [ 4043.1523 ],
       ...,
       [ -620.71484],
       [72390.42   ],
       [58873.496  ]], dtype=float32)

In [11]:
test = pd.read_csv('/content/dataset/test.csv')
test['Loan Sanction Amount (USD)'] = pred1



submission_1 = test[['Customer ID','Loan Sanction Amount (USD)']]
submission_1.set_index('Customer ID')

neg = submission_1[submission_1['Loan Sanction Amount (USD)'] < 0]
for i in neg['Loan Sanction Amount (USD)']:
  submission_1.replace(to_replace = i, value=0.00, inplace=True)
submission_1[submission_1['Loan Sanction Amount (USD)'] <= 0]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Customer ID,Loan Sanction Amount (USD)
5,C-2600,0.0
25,C-4949,0.0
28,C-23784,0.0
93,C-41801,0.0
109,C-36170,0.0
...,...,...
19930,C-26143,0.0
19963,C-41190,0.0
19972,C-22057,0.0
19988,C-43225,0.0


In [12]:
submission_1.to_csv('1.csv')
submission_1

,Customer ID,Loan Sanction Amount (USD)
0,C-26247,91595.726562
1,C-35067,65274.226562
2,C-34590,4043.152344
3,C-16668,47897.816406
4,C-12196,69078.242188
...,...,...
19995,C-9076,91384.984375
19996,C-17587,29461.355469
19997,C-46479,0.000000
19998,C-3099,72390.421875


## Model 2

In [13]:
model2 = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION,hyperparameter_template="benchmark_rank1")
model2.compile(metrics=['r2'])
with sys_pipes():
  model2.fit(x = tf_dataset)

464/464 [==============================] - 1s 2ms/step


[INFO kernel.cc:746] Start Yggdrasil model training
[INFO kernel.cc:747] Collect training examples
[INFO kernel.cc:392] Number of batches: 464
[INFO kernel.cc:393] Number of examples: 29659
[INFO kernel.cc:769] Dataset:
Number of records: 29659
Number of columns: 20

Number of columns by type:
	NUMERICAL: 20 (100%)

Columns:

NUMERICAL: 20 (100%)
	0: "Age" NUMERICAL mean:8.00443e-09 min:-1.37693 max:1.55144 sd:1
	1: "Co-Applicant" NUMERICAL mean:1.44495e-09 min:-4.88175 max:0.412622 sd:1
	2: "Credit_Score" NUMERICAL mean:7.89677e-11 min:-2.26367 max:2.21712 sd:1
	3: "Current_Loan_Expenses_(USD)" NUMERICAL mean:-1.66603e-10 min:-1.86244 max:15.6824 sd:1
	4: "Dependents" NUMERICAL mean:-9.64702e-09 min:-1.36103 max:12.763 sd:1
	5: "Expense_Type_1" NUMERICAL mean:3.38468e-08 min:-0.749932 max:1.33345 sd:1
	6: "Expense_Type_2" NUMERICAL mean:-2.59608e-08 min:-1.43324 max:0.697722 sd:1
	7: "Gender" NUMERICAL mean:-2.24078e-09 min:-1.00407 max:2.96841 sd:1
	8: "Has_Active_Credit_Card" NUMERI

In [14]:
pred2 = model2.predict(x_test)
pred2

array([[88215.49   ],
       [64764.047  ],
       [ 4383.6245 ],
       ...,
       [  581.74414],
       [67126.45   ],
       [68662.81   ]], dtype=float32)

In [15]:
test = pd.read_csv('/content/dataset/test.csv')
test['Loan Sanction Amount (USD)'] = pred2



submission_2 = test[['Customer ID','Loan Sanction Amount (USD)']]
submission_2.set_index('Customer ID')

neg2 = submission_2[submission_2['Loan Sanction Amount (USD)'] < 0]
for i in neg2['Loan Sanction Amount (USD)']:
  submission_2.replace(to_replace = i, value=0.00, inplace=True)
submission_2[submission_2['Loan Sanction Amount (USD)'] <= 0]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Customer ID,Loan Sanction Amount (USD)
132,C-35254,0.0
140,C-49153,0.0
143,C-28630,0.0
229,C-20553,0.0
238,C-22568,0.0
...,...,...
19728,C-7090,0.0
19746,C-7002,0.0
19839,C-46234,0.0
19906,C-45446,0.0


In [16]:
submission_2.to_csv('2.csv')
submission_2

,Customer ID,Loan Sanction Amount (USD)
0,C-26247,88215.492188
1,C-35067,64764.046875
2,C-34590,4383.624512
3,C-16668,60119.582031
4,C-12196,61008.628906
...,...,...
19995,C-9076,90869.648438
19996,C-17587,29271.894531
19997,C-46479,581.744141
19998,C-3099,67126.453125


### Take mean value of both predictions

In [17]:
one = pd.read_csv('/content/1.csv')
two = pd.read_csv('/content/2.csv')
m = (one['Loan Sanction Amount (USD)'] + two['Loan Sanction Amount (USD)']) / 2
m

0        89905.61000
1        65019.13700
2         4213.38840
3        54008.69900
4        65043.43500
            ...     
19995    91127.31700
19996    29366.62500
19997      290.87207
19998    69758.43500
19999    63768.15300
Name: Loan Sanction Amount (USD), Length: 20000, dtype: float64

In [18]:
one['Loan Sanction Amount (USD)'] = m
one.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
one.set_index('Customer ID')
one.to_csv('mean_sub.csv')
one

,Customer ID,Loan Sanction Amount (USD)
0,C-26247,89905.61000
1,C-35067,65019.13700
2,C-34590,4213.38840
3,C-16668,54008.69900
4,C-12196,65043.43500
...,...,...
19995,C-9076,91127.31700
19996,C-17587,29366.62500
19997,C-46479,290.87207
19998,C-3099,69758.43500


## ROC Score 0.813 , on 50% hidden test set